# School system, income and  venues data analysis of Boston

## 1. Introduction 
Boston is the capital and largest city of Massachusetts, United States. The city is the center of American higher education, health care and investment funds. It is one of the most educated cities in the United States. Its economic base is education, finance, medical care and technology. The City of Boston's Office of Neighborhood Services has designated 23 Neighborhoods as Allston, Back Bay, Bay Village, Beacon Hill, Brighton, Charlestown, Chinatown/Leather District, Dorchester, Downtown, East Boston, Fenway Kenmore, Hyde Park, Jamaica Plain, Mattapan, Mission Hill, North End, Roslindale, Roxbury, South Boston, South End, West End and West Roxbury. 
    
Due to the diversity of Boston, we would like to analyze the correlation between income and school district systems across the various districts of Boston, as well as the differences between these districts. We would also to analyze whether the characteristics of each district are related to their school district level and income level.
    
Our finding will be provided some information to people who want to rent a house in Boston to fit their lifestyle and education system they want.

## 2. Data
In this part, we will choose data from four parts.

I. <strong>The public school data of Boston.</strong> The data is obtained through https://www.kaggle.com/ndalziel/massachusetts-public-schools-data. This dataset compiles data from the following Massachusetts Department of Education reports:
Enrollment by Grade,
Enrollment by Selected Population,
Enrollment by Race/Gender,
Class Size by Gender and Selected Populations,
Teacher Salaries,
Per Pupil Expenditure,
Graduation Rates,
Graduates Attending Higher Ed,
Advanced Placement Participation,
Advanced Placement Performance,
SAT Performance,
MCAS Achievement Results,
Accountability Report.  In each case, the data is the latest available data as of August 2017.

II. <strong>The income data of Boston.</strong> The data is obtained through https://en.wikipedia.org/wiki/Boston. Data is from the 2008–2012 American Community Survey 5-Year Estimates. 

III. <strong>The Administrative District data of Boston. </strong> The data is obtained through https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/table/?refine.state=MA. This database was composed using ZIP code gazetteers from the US Census Bureau from 1999 and 2000. The latitude and longitude given for each ZIP code is typically (though not always) the geographic centroid of the ZIP code. 

IV. <strong>The most common venues data of Boston</strong>. Forsquare API will be used to get the most common venues of Boston. 

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import csv

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 2.1 Public school data 

In [2]:
df1=pd.read_csv('MA_Public_Schools_2017.csv')

In [3]:
df1.head(5)

,School Code,School Name,School Type,Function,Contact Name,Address 1,Address 2,Town,State,Zip,Phone,Fax,Grade,District Name,District Code,PK_Enrollment,K_Enrollment,1_Enrollment,2_Enrollment,3_Enrollment,4_Enrollment,5_Enrollment,6_Enrollment,7_Enrollment,8_Enrollment,9_Enrollment,10_Enrollment,11_Enrollment,12_Enrollment,SP_Enrollment,TOTAL_Enrollment,First Language Not English,% First Language Not English,English Language Learner,% English Language Learner,Students With Disabilities,% Students With Disabilities,High Needs,% High Needs,Economically Disadvantaged,% Economically Disadvantaged,% African American,% Asian,% Hispanic,% White,% Native American,"% Native Hawaiian, Pacific Islander","% Multi-Race, Non-Hispanic",% Males,% Females,Total # of Classes,Average Class Size,Number of Students,Salary Totals,Average Salary,FTE Count,In-District Expenditures,Total In-district FTEs,Average In-District Expenditures per Pupil,Total Expenditures,Total Pupil FTEs,Average Expenditures per Pupil,# in Cohort,% Graduated,% Still in School,% Non-Grad Completers,% GED,% Dropped Out,% Permanently Excluded,High School Graduates (#),Attending Coll./Univ. (#),% Attending College,% Private Two-Year,% Private Four-Year,% Public Two-Year,% Public Four-Year,% MA Community College,% MA State University,% UMass,AP_Test Takers,AP_Tests Taken,AP_One Test,AP_Two Tests,AP_Three Tests,AP_Four Tests,AP_Five or More Tests,AP_Score=1,AP_Score=2,AP_Score=3,AP_Score=4,AP_Score=5,% AP_Score 1-2,% AP_Score 3-5,SAT_Tests Taken,Average SAT_Reading,Average SAT_Writing,Average SAT_Math,MCAS_3rdGrade_Math_P+A #,% MCAS_3rdGrade_Math_P+A,MCAS_3rdGrade_Math_A #,% MCAS_3rdGrade_Math_A,MCAS_3rdGrade_Math_P #,% MCAS_3rdGrade_Math_P,MCAS_3rdGrade_Math_NI #,% MCAS_3rdGrade_Math_NI,MCAS_3rdGrade_Math_W/F #,% MCAS_3rdGrade_Math_W/F,MCAS_3rdGrade_Math_Stud. Incl. #,MCAS_3rdGrade_Math_CPI,MCAS_3rdGrade_Math_SGP,MCAS_3rdGrade_Math_Incl. in SGP(#),MCAS_4thGrade_Math_P+A #,% MCAS_4thGrade_Math_P+A,MCAS_4thGrade_Math_A #,% MCAS_4thGrade_Math_A,MCAS_4thGrade_Math_P #,% MCAS_4thGrade_Math_P,MCAS_4thGrade_Math_NI #,% MCAS_4thGrade_Math_NI,MCAS_4thGrade_Math_W/F #,% MCAS_4thGrade_Math_W/F,MCAS_4thGrade_Math_Stud. Incl. #,MCAS_4thGrade_Math_CPI,MCAS_4thGrade_Math_SGP,MCAS_4thGrade_Math_Incl. in SGP(#),MCAS_5thGrade_Math_P+A #,% MCAS_5thGrade_Math_P+A,MCAS_5thGrade_Math_A #,% MCAS_5thGrade_Math_A,MCAS_5thGrade_Math_P #,% MCAS_5thGrade_Math_P,MCAS_5thGrade_Math_NI #,% MCAS_5thGrade_Math_NI,MCAS_5thGrade_Math_W/F #,% MCAS_5thGrade_Math_W/F,MCAS_5thGrade_Math_Stud. Incl. #,MCAS_5thGrade_Math_CPI,MCAS_5thGrade_Math_SGP,MCAS_5thGrade_Math_Incl. in SGP(#),MCAS_6thGrade_Math_P+A #,% MCAS_6thGrade_Math_P+A,MCAS_6thGrade_Math_A #,% MCAS_6thGrade_Math_A,MCAS_6thGrade_Math_P #,% MCAS_6thGrade_Math_P,MCAS_6thGrade_Math_NI #,% MCAS_6thGrade_Math_NI,MCAS_6thGrade_Math_W/F #,% MCAS_6thGrade_Math_W/F,MCAS_6thGrade_Math_Stud. Incl. #,MCAS_6thGrade_Math_CPI,MCAS_6thGrade_Math_SGP,MCAS_6thGrade_Math_Incl. in SGP(#),MCAS_7thGrade_Math_P+A #,% MCAS_7thGrade_Math_P+A,MCAS_7thGrade_Math_A #,% MCAS_7thGrade_Math_A,MCAS_7thGrade_Math_P #,% MCAS_7thGrade_Math_P,MCAS_7thGrade_Math_NI #,% MCAS_7thGrade_Math_NI,MCAS_7thGrade_Math_W/F #,% MCAS_7thGrade_Math_W/F,MCAS_7thGrade_Math_Stud. Incl. #,MCAS_7thGrade_Math_CPI,MCAS_7thGrade_Math_SGP,MCAS_7thGrade_Math_Incl. in SGP(#),MCAS_8thGrade_Math_P+A #,% MCAS_8thGrade_Math_P+A,MCAS_8thGrade_Math_A #,% MCAS_8thGrade_Math_A,MCAS_8thGrade_Math_P #,% MCAS_8thGrade_Math_P,MCAS_8thGrade_Math_NI #,% MCAS_8thGrade_Math_NI,MCAS_8thGrade_Math_W/F #,% MCAS_8thGrade_Math_W/F,MCAS_8thGrade_Math_Stud. Incl. #,MCAS_8thGrade_Math_CPI,MCAS_8thGrade_Math_SGP,MCAS_8thGrade_Math_Incl. in SGP(#),MCAS_10thGrade_Math_P+A #,% MCAS_10thGrade_Math_P+A,MCAS_10thGrade_Math_A #,% MCAS_10thGrade_Math_A,MCAS_10thGrade_Math_P #,% MCAS_10thGrade_Math_P,MCAS_10thGrade_Math_NI #,% MCAS_10thGrade_Math_NI,MCAS_10thGrade_Math_W/F #,% MCAS_10thGrade_Math_W/F,MCAS_10thGrade_Math_Stud. Incl. #,MCAS_10thGrade_Mat

In [4]:
df1.shape

(1861, 302)

 Create a school dataframe, here we only use one paramenter <strong>"Progress and Performance Index (PPI) - All Students"</strong> to indicate the school quality. 

In [5]:
# define the dataframe columns
column_names = ['SchoolName','Town', 'Zip', 'PPI'] 

# instantiate the dataframe
school_df1 = pd.DataFrame(columns=column_names)

In [6]:
for index, row in df1.iterrows():
         school_df1 = school_df1.append({'SchoolName':df1.iloc[index]['School Name'],
                                      'Town': df1.iloc[index]['Town'],
                                      'Zip': '0'+str(df1.iloc[index]['Zip']),
                                     'PPI': df1.iloc[index]['Progress and Performance Index (PPI) - All Students']
                                        }, ignore_index=True)

In [7]:
school_df1.head()

,SchoolName,Town,Zip,PPI
0,Abington High,Abington,02351,76.0
1,Beaver Brook Elementary School,Abington,02351,69.0
2,Center Elementary School,Abington,02351,NaN
3,Frolio Middle School,Abington,02351,63.0
4,Woodsdale Elementary School,Abington,02351,65.0


In [8]:
school_df2=school_df1.groupby('Zip').mean()

In [9]:
school_df2.head()

,PPI
Zip,
01001,58.400000
01002,62.666667
01005,57.000000
01007,65.500000
01010,83.000000


In [10]:
school_df2.reset_index(level=0, inplace=True)

In [11]:
school_df2.head(5)

,Zip,PPI
0,01001,58.400000
1,01002,62.666667
2,01005,57.000000
3,01007,65.500000
4,01010,83.000000


In [12]:
school_df2.shape

(433, 2)

## 2.2 The income data of Boston

In [13]:
r = requests.get("https://en.wikipedia.org/wiki/Boston")
soup = BeautifulSoup(r.content, 'lxml')

 Scraping a Wikipedia table with Beautiful Soup

In [14]:
# Load article, turn into soup and get the tables.
tables = soup.find_all('table', class_='sortable')

In [15]:
# Search through the tables for the one with the headings we want.
for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings[:2] == ['Rank', 'ZIP code (ZCTA)']:
        break

In [16]:
csv_file=open('Boston_wiki_01.csv','w')
csv_writer=csv.writer(csv_file)
csv_writer.writerow(headings)

111

In [17]:
for item in table.tbody.find_all('tr'):
    text1=item.text
    data1=text1.split("\n")[1]
    data2=text1.split("\n")[3].split()[0]
    data3=text1.split("\n")[5]
    data4=text1.split("\n")[7]
    data5=text1.split("\n")[9]
    data6=text1.split("\n")[11]
    data7=text1.split("\n")[13]
    
    
    csv_writer.writerow([data1, data2, data3, data4, data5, data6, data7])

In [18]:
csv_file.close()

In [19]:
df2=pd.read_csv('Boston_wiki_01.csv')

In [20]:
df2.dropna(axis=0, inplace=True)
df2.shape

(31, 7)

In [21]:
df2.head(5)

,Rank,ZIP code (ZCTA),Per capitaincome,Medianhouseholdincome,Medianfamilyincome,Population,Number ofhouseholds
0,Rank,ZIP,Per capitaincome,Medianhouseholdincome,Medianfamilyincome,Population,Number ofhouseholds
1,1,02110,"$152,007","$123,795","$196,518","1,486",981
2,2,02199,"$151,060","$107,159","$146,786","1,290",823
3,3,02210,"$93,078","$111,061","$223,411","1,905","1,088"
4,4,02109,"$88,921","$128,022","$162,045","4,277","2,190"


In [22]:
df2.dtypes

Rank                     object
ZIP code (ZCTA)          object
Per capitaincome         object
Medianhouseholdincome    object
Medianfamilyincome       object
Population               object
Number ofhouseholds      object
dtype: object

We only use <strong>Medicanhouseholdincome</strong> as an indicator for income of Boston 

In [23]:
df2.drop([0], inplace=True)

In [24]:
df2.drop(columns=['Rank'], inplace=True)

In [25]:
df2.drop(columns=['Per capitaincome', 'Medianfamilyincome', 'Population', 'Number ofhouseholds'], inplace=True)

In [26]:
df2.rename(columns={'ZIP code (ZCTA)': "Zip"}, inplace=True)

In [27]:
df2 = df2.reset_index(drop=True)


In [28]:
df2.head()

,Zip,Medianhouseholdincome
0,02110,"$123,795"
1,02199,"$107,159"
2,02210,"$111,061"
3,02109,"$128,022"
4,02116,"$87,630"


In [29]:
df2.shape

(30, 2)

## 3.3 The Administrative District data of Boston

"https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/table/?refine.state=MA" downland data


In [30]:
df4=pd.read_csv('us-zip-code-latitude-and-longitude.csv')

In [31]:
df4.shape

(43191, 1)

In [32]:
df4.head(4)

,Zip;City;State;Latitude;Longitude;Timezone;Daylight savings time flag;geopoint
49654;Leland;MI;45.023384;-85.75697;-5;1;45.023384,-85.756970
50450;Lake Mills;IA;43.418877;-93.52078;-6;1;43.418877,-93.520780
52594;Unionville;IA;40.833045;-92.6867;-6;1;40.833045,-92.686700
54307;Green Bay;WI;44.460064;-88.007382;-6;1;44.460064,-88.007382


In [33]:
df4.reset_index(level=0, inplace=True)
df4.head()

,index,Zip;City;State;Latitude;Longitude;Timezone;Daylight savings time flag;geopoint
0,49654;Leland;MI;45.023384;-85.75697;-5;1;45.02...,-85.756970
1,50450;Lake Mills;IA;43.418877;-93.52078;-6;1;4...,-93.520780
2,52594;Unionville;IA;40.833045;-92.6867;-6;1;40...,-92.686700
3,54307;Green Bay;WI;44.460064;-88.007382;-6;1;4...,-88.007382
4,54812;Barron;WI;45.39701;-91.86337;-6;1;45.39701,-91.863370


In [34]:
df4.shape

(43191, 2)

In [35]:
df4.drop(['Zip;City;State;Latitude;Longitude;Timezone;Daylight savings time flag;geopoint'], axis=1, inplace=True)

In [36]:
df4.head()

,index
0,49654;Leland;MI;45.023384;-85.75697;-5;1;45.02...
1,50450;Lake Mills;IA;43.418877;-93.52078;-6;1;4...
2,52594;Unionville;IA;40.833045;-92.6867;-6;1;40...
3,54307;Green Bay;WI;44.460064;-88.007382;-6;1;4...
4,54812;Barron;WI;45.39701;-91.86337;-6;1;45.39701


In [37]:
#df4.iloc[1][0].split(';')

In [38]:
# define the dataframe columns
column_names = ['Zip','city', 'state', 'Latitude', 'Longitude'] 

# instantiate the dataframe
MA_df = pd.DataFrame(columns=column_names)

In [39]:
MA_df

,Zip,city,state,Latitude,Longitude


In [40]:
item=0
for index, row in df4.iterrows():
    if (df4.iloc[index][0].split(';')[2] == 'MA'):
         MA_df = MA_df.append({'Zip':df4.iloc[index][0].split(';')[0],
                               'city': df4.iloc[index][0].split(';')[1],
                               'state': df4.iloc[index][0].split(';')[2],
                                'Latitude': float(df4.iloc[index][0].split(';')[3]),
                                'Longitude': float(df4.iloc[index][0].split(';')[4])}, ignore_index=True)

In [41]:
MA_df.head()

,Zip,city,state,Latitude,Longitude
0,01888,Woburn,MA,42.446396,-71.459405
1,02459,Newton Center,MA,42.321197,-71.193000
2,02454,Waltham,MA,42.356719,-71.250479
3,01068,Oakham,MA,42.349769,-72.044130
4,02702,Assonet,MA,41.792192,-71.064460


In [42]:
MA_df.shape

(766, 5)

## 3.4 Join tables 

### 3.4.1 Join school table and income table with the same zip code

In [43]:
school_df2.head()

,Zip,PPI
0,01001,58.400000
1,01002,62.666667
2,01005,57.000000
3,01007,65.500000
4,01010,83.000000


In [44]:
school_df2.shape

(433, 2)

In [45]:
df2.head()

,Zip,Medianhouseholdincome
0,02110,"$123,795"
1,02199,"$107,159"
2,02210,"$111,061"
3,02109,"$128,022"
4,02116,"$87,630"


In [46]:
df2.shape

(30, 2)

In [47]:
df_join1=df2.join(school_df2.set_index('Zip'), on='Zip')

In [48]:
df_join1.head()

,Zip,Medianhouseholdincome,PPI
0,02110,"$123,795",NaN
1,02199,"$107,159",NaN
2,02210,"$111,061",NaN
3,02109,"$128,022",NaN
4,02116,"$87,630",55.0


In [49]:
df_join1.shape

(30, 3)

### 3.4.3 Jion the MA_df coordinate table

In [50]:
df_join2=df_join1.join(MA_df.set_index('Zip'), on='Zip')
df_join2.head()

,Zip,Medianhouseholdincome,PPI,city,state,Latitude,Longitude
0,02110,"$123,795",NaN,Boston,MA,42.356532,-71.05365
1,02199,"$107,159",NaN,Boston,MA,42.347130,-71.08234
2,02210,"$111,061",NaN,Boston,MA,42.347974,-71.04463
3,02109,"$128,022",NaN,Boston,MA,42.361477,-71.05417
4,02116,"$87,630",55.0,Boston,MA,42.349622,-71.07372


In [51]:
df_join2.shape

(30, 7)

In [52]:
Boston_df=df_join2

In [53]:
Boston_df.drop(['city', 'state'], axis=1, inplace=True)

In [54]:
Boston_df.head(1)

,Zip,Medianhouseholdincome,PPI,Latitude,Longitude
0,02110,"$123,795",NaN,42.356532,-71.05365


## Use geopy library to get the latitude and longitude values of Toronto.

In [55]:
address = 'Boston, MA'

geolocator = Nominatim(user_agent="boston_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Boston City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Boston City are 42.3602534, -71.0582912.


In [56]:
# create map of Boston using latitude and longitude values
map_Boston = folium.Map(location=[latitude, longitude], zoom_start=10)


    
map_Boston

Define Foursquare Credentials and Version

In [57]:
CLIENT_ID = '0UKYDUTATAWZOK1IPONWZ4VBIOHOXJOTUZXJTEIJSKRW3IAS' # your Foursquare ID
CLIENT_SECRET = 'OFUEVSLXLFHJONAEGDZGYE3Z34PKCRPMOVMOSA23CCWD1PX2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0UKYDUTATAWZOK1IPONWZ4VBIOHOXJOTUZXJTEIJSKRW3IAS
CLIENT_SECRET:OFUEVSLXLFHJONAEGDZGYE3Z34PKCRPMOVMOSA23CCWD1PX2


In [58]:
zip_latitude = Boston_df.loc[0, 'Latitude'] # neighborhood latitude value
zip_longitude = Boston_df.loc[0, 'Longitude'] # neighborhood longitude value

zip_name = Boston_df.loc[0, 'Zip'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(zip_name, 
                                                               zip_latitude, 
                                                               zip_longitude))

Latitude and longitude values of 02110 are 42.356532, -71.05365.


Now, let's get the top 100 venues that are in "02110" within a radius of 500 meters.

In [59]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    zip_latitude, 
    zip_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=0UKYDUTATAWZOK1IPONWZ4VBIOHOXJOTUZXJTEIJSKRW3IAS&client_secret=OFUEVSLXLFHJONAEGDZGYE3Z34PKCRPMOVMOSA23CCWD1PX2&v=20180605&ll=42.356532,-71.05365&radius=500&limit=100'

Send the GET request and examine the resutls

In [60]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d1284f86e46500039e7f26e'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Financial District',
  'headerFullLocation': 'Financial District, Boston',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 160,
  'suggestedBounds': {'ne': {'lat': 42.361032004500004,
    'lng': -71.04757177954377},
   'sw': {'lat': 42.3520319955, 'lng': -71.05972822045624}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5b05fb67f193c0002cbd9455',
       'name': 'Trillium Garden On The Greenway',
       'location': {'address': 'Atlantic Ave',
        'crossStreet': 'High St',
        'lat': 42.35653994232763,
   

In [61]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [62]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Trillium Garden On The Greenway,Beer Garden,42.356540,-71.051100
1,Kane's Donuts,Donut Shop,42.356209,-71.052895
2,The Rose Kennedy Greenway,Park,42.358227,-71.052055
3,Equinox Franklin Street,Gym,42.356074,-71.054484
4,Post Office Square,Park,42.356340,-71.055686


In [63]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))


100 venues were returned by Foursquare.


In [64]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Zip', 
                  'Zip Latitude', 
                  'Zip Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [65]:
Boston_venues = getNearbyVenues(names=Boston_df['Zip'],
                                   latitudes=Boston_df['Latitude'],
                                   longitudes=Boston_df['Longitude']
                                  )

02110
02199
02210
02109
02116
02108
02114
02111
02129
02467
02113
02132
02118
02130
02127
02135
02131
02136
02134
02128
02122
02124
02125
02163
02115
02126
02215
02119
02121
02120


In [66]:
print(Boston_venues.shape)
Boston_venues.head()

(1469, 7)


,Zip,Zip Latitude,Zip Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,02110,42.356532,-71.05365,Trillium Garden On The Greenway,42.356540,-71.051100,Beer Garden
1,02110,42.356532,-71.05365,Kane's Donuts,42.356209,-71.052895,Donut Shop
2,02110,42.356532,-71.05365,The Rose Kennedy Greenway,42.358227,-71.052055,Park
3,02110,42.356532,-71.05365,Equinox Franklin Street,42.356074,-71.054484,Gym
4,02110,42.356532,-71.05365,Post Office Square,42.356340,-71.055686,Park


In [67]:
Boston_venues.groupby('Zip').count()

,Zip Latitude,Zip Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Zip,,,,,,
02108,100,100,100,100,100,100
02109,100,100,100,100,100,100
02110,100,100,100,100,100,100
02111,100,100,100,100,100,100
02113,100,100,100,100,100,100
02114,51,51,51,51,51,51
02115,38,38,38,38,38,38
02116,100,100,100,100,100,100
02118,55,55,55,55,55,55


In [68]:
print('There are {} uniques categories.'.format(len(Boston_venues['Venue Category'].unique())))

There are 221 uniques categories.


Analyze Each Neighborhood

In [69]:
# one hot encoding
Boston_onehot = pd.get_dummies(Boston_venues[['Venue Category']], prefix="", prefix_sep="")

# add zip column back to dataframe
Boston_onehot['Zip'] = Boston_venues['Zip'] 

# move zip column to the first column
fixed_columns = [Boston_onehot.columns[-1]] + list(Boston_onehot.columns[:-1])
Boston_onehot = Boston_onehot[fixed_columns]

Boston_onehot.head()

,Zip,ATM,Accessories Store,African Restaurant,Airport,American Restaurant,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Café,Cambodian Restaurant,Caribbean Restaurant,Cemetery,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Insurance Office,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Library,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Other Repair Shop,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Ski Chalet,Smoke Shop,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,02110,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,02110,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [70]:
Boston_onehot.shape


(1469, 222)

Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [71]:
Boston_grouped = Boston_onehot.groupby('Zip').mean().reset_index()
Boston_grouped

,Zip,ATM,Accessories Store,African Restaurant,Airport,American Restaurant,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Café,Cambodian Restaurant,Caribbean Restaurant,Cemetery,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Insurance Office,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Library,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Other Repair Shop,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Ski Chalet,Smoke Shop,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,02108,0.000000,0.000000,0.000000,0.000000,0.050000,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.000000,0.00,0.000000,0.000000,0.020000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.00000,0.00,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.01,0.020000,0.00,0.010000,0.040000,0.000000,0.000000,0.00,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.000000,0.00,0.010000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.020000,0.000000,0.000000,0.010000,0.010000,0.010000,0.010000,0.000000,0.000000,0.03000,0.0

In [72]:
Boston_grouped.shape

(30, 222)

 Print each neighborhood along with the top 5 most common venues

In [73]:
num_top_venues = 5

for hood in Boston_grouped['Zip']:
    print("----"+hood+"----")
    temp = Boston_grouped[Boston_grouped['Zip'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----02108----
                 venue  freq
0  American Restaurant  0.05
1       Sandwich Place  0.05
2          Pizza Place  0.04
3          Coffee Shop  0.04
4                Plaza  0.04


----02109----
                venue  freq
0  Italian Restaurant  0.22
1  Seafood Restaurant  0.11
2              Bakery  0.06
3       Historic Site  0.04
4                Park  0.04


----02110----
                 venue  freq
0   Seafood Restaurant  0.06
1          Coffee Shop  0.06
2  American Restaurant  0.06
3       Sandwich Place  0.05
4                Hotel  0.04


----02111----
                venue  freq
0  Chinese Restaurant  0.13
1    Asian Restaurant  0.08
2              Bakery  0.07
3    Sushi Restaurant  0.04
4         Coffee Shop  0.04


----02113----
                venue  freq
0  Italian Restaurant  0.39
1              Bakery  0.06
2         Pizza Place  0.05
3                Park  0.05
4  Seafood Restaurant  0.04


----02114----
                venue  freq
0  Italian Restaurant  0.0

write a function to sort the venues in descending order.¶


In [74]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


create the new dataframe and display the top 10 venues for each neighborhood.¶


In [75]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Zip']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
zip_venues_sorted = pd.DataFrame(columns=columns)
zip_venues_sorted['Zip'] = Boston_grouped['Zip']

for ind in np.arange(Boston_grouped.shape[0]):
    zip_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Boston_grouped.iloc[ind, :], num_top_venues)

zip_venues_sorted.head()

,Zip,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,02108,American Restaurant,Sandwich Place,Coffee Shop,Plaza,Pizza Place,Italian Restaurant,Restaurant,Steakhouse,New American Restaurant,Historic Site
1,02109,Italian Restaurant,Seafood Restaurant,Bakery,Historic Site,Park,Pizza Place,Café,Pub,Hotel,Sandwich Place
2,02110,Seafood Restaurant,Coffee Shop,American Restaurant,Sandwich Place,Historic Site,Hotel,Park,Salad Place,Steakhouse,Aquarium
3,02111,Chinese Restaurant,Asian Restaurant,Bakery,Theater,Coffee Shop,Sushi Restaurant,Café,Performing Arts Venue,Gym,Dive Bar
4,02113,Italian Restaurant,Bakery,Park,Pizza Place,Coffee Shop,Seafood Restaurant,Café,Sandwich Place,Market,Yoga Studio


Run k-means to cluster the neighborhood into 5 clusters.

In [76]:
# set number of clusters
kclusters = 5

Boston_grouped_clustering = Boston_grouped.drop('Zip', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Boston_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 0, 0, 2, 0, 0, 0, 0, 1], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [77]:
# add clustering labels
zip_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Boston_merged = Boston_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Boston_merged = Boston_merged.join(zip_venues_sorted.set_index('Zip'), on='Zip')

Boston_merged.head() # check the last columns!

,Zip,Medianhouseholdincome,PPI,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,02110,"$123,795",NaN,42.356532,-71.05365,0,Seafood Restaurant,Coffee Shop,American Restaurant,Sandwich Place,Historic Site,Hotel,Park,Salad Place,Steakhouse,Aquarium
1,02199,"$107,159",NaN,42.347130,-71.08234,0,Coffee Shop,American Restaurant,Clothing Store,Hotel,Ice Cream Shop,Spa,Italian Restaurant,Seafood Restaurant,Bar,Dessert Shop
2,02210,"$111,061",NaN,42.347974,-71.04463,0,Restaurant,Hotel,Coffee Shop,American Restaurant,Italian Restaurant,Donut Shop,Asian Restaurant,Bar,Steakhouse,Mexican Restaurant
3,02109,"$128,022",NaN,42.361477,-71.05417,2,Italian Restaurant,Seafood Restaurant,Bakery,Historic Site,Park,Pizza Place,Café,Pub,Hotel,Sandwich Place
4,02116,"$87,630",55.0,42.349622,-71.07372,0,Spa,Gym / Fitness Center,Sandwich Place,American Restaurant,Italian Restaurant,Gym,Cosmetics Shop,Coffee Shop,Jewelry Store,Mexican Restaurant


In [78]:
Boston_merged['Cluster Labels'].value_counts()

0    20
4     6
2     2
3     1
1     1
Name: Cluster Labels, dtype: int64

In [79]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Boston_merged['Latitude'], Boston_merged['Longitude'], Boston_merged['Zip'], Boston_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 3. Methodology 
Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, and what machine learnings were used and why.


## 4. Results 
Results section where you discuss the results.


## 5. Discussion 
Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.


## 6. Conclusion 
Conclusion section where you conclude the report.